In [ ]:
import cv2
import pandas as pd
import numpy as np
import imageio
import os
from IPython.display import Image, display

In [ ]:
vs = cv2.VideoCapture('volleyball_match.mp4')

In [ ]:
MIN_AREA = 200 
MAX_AREA = 900

fgbg = cv2.createBackgroundSubtractorMOG2(history=20, varThreshold=50, detectShadows=True)

trajectory = []        
max_points = 2     

while True:
    ret, frame = vs.read()
    if not ret:
        break

    fgmask = fgbg.apply(frame)

    _, fgmask = cv2.threshold(fgmask, 250, 255, cv2.THRESH_BINARY)

    fgmask = cv2.erode(fgmask, None, iterations=1)
    fgmask = cv2.dilate(fgmask, None, iterations=2)

    cnts, _ = cv2.findContours(
        fgmask.copy(),
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE
    )
    
    ball_candidate = None

    for c in cnts:
        area = cv2.contourArea(c)
        
        if area < MIN_AREA or area > MAX_AREA:
            continue
            
        x, y, w, h = cv2.boundingRect(c)
        aspect_ratio = float(w) / h
        
        if aspect_ratio < 0.7 or aspect_ratio > 1.5:
            continue

        perimeter = cv2.arcLength(c, True)
        if perimeter == 0:
            continue

        circularity = (4 * np.pi * area) / (perimeter ** 2)
        if circularity < 0.7:
            continue

        cx = int(x + w/2)
        cy = int(y + h/2)
        r  = int(w/2)

        ball_candidate = (cx, cy, r)

        trajectory.append((cx, cy))

        if len(trajectory) > max_points:
            trajectory.pop(0)

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, "Ball", (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


    for i in range(1, len(trajectory)):
        x1, y1 = trajectory[i - 1]
        x2, y2 = trajectory[i]

        cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

    cv2.imshow("Detect", frame)

    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

vs.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

VIDEO_PATH = 'volleyball_match.mp4'
MODEL_SIZE = 'yolov8n.pt'

NET_Y_LEVEL = 420 

model = YOLO(MODEL_SIZE)

court_polygon = np.array([
    [300, 275],   # Top-Left
    [980, 263],   # Top-Right
    [1250, 700],  # Bottom-Right 
    [75, 700]     # Bottom-Left
], np.int32)

cap = cv2.VideoCapture(VIDEO_PATH)

while True:
    ret, frame = cap.read()
    if not ret: break

    height, width, _ = frame.shape
    
    results = model.predict(frame, conf=0.3, classes=[0], verbose=False)
    boxes = results[0].boxes

    far_team_count = 0 
    near_team_count = 0 

    cv2.polylines(frame, [court_polygon], isClosed=True, color=(255, 255, 0), thickness=2)
    
    cv2.line(frame, (0, NET_Y_LEVEL), (width, NET_Y_LEVEL), (255, 255, 255), 2)

    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
        
        feet_x = int((x1 + x2) / 2)
        feet_y = int(y2)

        is_inside = cv2.pointPolygonTest(court_polygon, (feet_x, feet_y), False)

        if is_inside >= 0:

            if feet_y < NET_Y_LEVEL:
                far_team_count += 1
                color = (0, 255, 255) # Yellow Box
                label = "Far"
            else:

                near_team_count += 1
                color = (0, 0, 255) # Red Box
                label = "Near"
            

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

    overlay = frame.copy()
    cv2.rectangle(overlay, (0, 0), (width, 80), (0, 0, 0), -1)
    frame = cv2.addWeighted(overlay, 0.4, frame, 0.6, 0)

    cv2.putText(frame, f"Far Team: {far_team_count}", (50, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    
    cv2.putText(frame, f"Near Team: {near_team_count}", (width - 350, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Team Counter", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()